<a href="https://colab.research.google.com/github/suryaph971/AWSDELearning/blob/master/2B_langchain_embeddings_pinecone_similarity_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -r requirements.txt

  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 2.0 MB/s eta 0:00:00
Using cached langchain_core-0.2.43-py3-none-any.whl (397 kB)
Using cached langsmith-0.1.147-py3-none-any.whl (311 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.4.10
    Uninstalling langsmith-

In [11]:
pip install pinecone

In [12]:
pip install langchain-cohere

In [1]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)

True

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('churchill_speech.txt') as f:
    churchill_speech = f.read()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100,
        chunk_overlap=20,
        length_function=len

    )


In [3]:
chunks = text_splitter.create_documents([churchill_speech])
print(len(chunks))

300


#Embedding Cost

In [4]:
def embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-3-small')
    total_tokens=sum([len(enc.encode(page.page_content)) for page in texts])
    print(f"total tokens:{total_tokens}")
    print(f"Embedding cose in USD : {total_tokens/1000*0.00002:.6f}")

embedding_cost(chunks)

total tokens:4820
Embedding cose in USD : 0.000096


#creating Embeddings

In [33]:
from langchain_cohere import CohereEmbeddings
embeddings = CohereEmbeddings(model='embed-english-v3.0')

In [14]:
vector = embeddings.embed_query(chunks[0].page_content)
print(vector)

[0.093933105, 0.007598877, 0.03390503, -0.014915466, 0.048553467, -0.0345459, 0.07659912, -0.019073486, -3.6656857e-05, 0.0129470825, 0.0009303093, 0.018081665, -0.04196167, -0.038482666, -0.01108551, -0.015930176, -0.030319214, 0.026779175, -0.016220093, -0.03869629, 0.05441284, -0.05645752, -0.06903076, 0.025634766, 0.017959595, -0.030853271, -0.056915283, 0.036895752, -0.03189087, -0.015686035, -0.004058838, 0.057800293, 0.016464233, -0.010383606, 0.0062446594, -0.04724121, -0.0014181137, -0.014411926, 0.031585693, -0.021011353, -0.004032135, -0.0071105957, -0.029800415, -0.06744385, -0.057006836, 0.018203735, -0.03479004, 0.026809692, 0.008163452, 0.017807007, 0.008659363, 0.008102417, -0.005241394, 0.025482178, -0.021072388, -0.004558563, -0.011734009, 0.02470398, 0.023788452, 7.790327e-05, -0.011833191, 0.023254395, 0.01550293, 0.018417358, -0.007801056, -0.008842468, 0.033050537, 0.0027999878, 0.050048828, 0.020599365, 0.019256592, 0.04586792, 0.02078247, -0.03274536, -0.0135498

#Inserting the Embeddings into a Pinecone Index

In [28]:
import os
import pinecone
from langchain.vectorstores import Pinecone
pc = pinecone.Pinecone()


In [29]:
#deleting all indexes

indexes = pc.list_indexes().names()
for index in indexes:
    print(f'deleting index : {index}')
    pc.delete_index(index)
    print("Done")


deleting index : churchill-speech
Done


In [30]:
#Creating index serverlessSpec
from pinecone import ServerlessSpec
index_name = 'churchill-speech'
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension = 1536,
        metric = 'cosine',
        spec = ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
    print("Index created")
else:
    print("Index already exists")

Index created


In [26]:
pip install langchain cohere pinecone-client


In [31]:
#processing the input documents,generating the embeddings using the provided cohereEmbeddings
#inserting the embeddings into the index and returning a pinecone vector store object

vector_stores = Pinecone.from_documents(chunks,embeddings,index_name=index_name)


AttributeError: module 'pinecone' has no attribute 'Index'

In [19]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

#Answering Questions to similarity Search

In [ ]:
query='Where Should we fight?'
result = vector_stores.similarity_search(query)
print(result)

In [ ]:
for i in result:
    print(i.page_content)
    print('-'*50)


#Answering in Natural Language using an LLM

In [22]:
from langchain.chains import RetrievalQA
from langchain_cohere import ChatCohere
llm = ChatCohere(model='command-nightly')
retriever = vector_stores.as_retriever(search_type='similarity',search_kwargs={'k': 3})
chain = RetrievalQA.from_chain_type(llm=llm,chain_type='stuff',retriever=retriever)

NameError: name 'vector_stores' is not defined

In [23]:
query = 'Answer only from the provided input. Where should we fight?'
query=chain.invoke(query)
print(answer)

NameError: name 'chain' is not defined

In [24]:
query = 'Who was the king of Belgium at that time?'
answer = chain.invoke(query)
print(answer)

NameError: name 'chain' is not defined

In [25]:
query = 'What about the French Armies??'
answer = chain.invoke(query)
print(answer)

NameError: name 'chain' is not defined